In [1]:
{-# LANGUAGE TypeOperators, ParallelListComp, ScopedTypeVariables, Rank2Types, UnicodeSyntax #-}
:opt no-lint

In [2]:
import Data.VectorSpace
import Math.LinearMap.Category
import Prelude ()
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
import Data.Semigroup ((<>))
type ℝ = Double

In [3]:
import System.Random
import Control.Monad (replicateM)
randomPoints <- replicateM 100 $ randomRIO (0,10) :: IO [ℝ]
randomDeviations <- cycle <$> replicateM 165 `id` randomRIO (-0.1, 0.1) :: IO [ℝ]

In [4]:
mapM_ print (
 linearRegressionW euclideanNorm
    (\x -> lfun $ \(a,b) -> a*x + b)
 <$> [ [(0,1), (1,2), (2,3)]
     , [(-2,6), (0,2), (1,0)]
     , [(x, 14.3*x + 2.9 + δy) | x<-randomPoints
                               | δy<-randomDeviations]
     :: [(ℝ,ℝ)] ])

(1.0,1.0)
(-2.0,1.9999999999999996)
(14.299658115052473,2.904942191502414)

In [5]:
import Data.List (sort)
newtype Take = Take (∀ a . Int -> [a] -> [a])
takeSome' <- do
   randomIndices <- replicateM 10000 $ randomRIO (0, 10000)
   return $ Take (\n xs -> let l = length xs
                               go (i₀:i₁:is) xs'
                                | (e:es, r) <- splitAt (i₁-i₀) xs'  = e : go (i₁:is) r
                               go (i₀:is) (x₀:xs')  = x₀ : go is xs'
                               go _ _ = []
                           in go (sort . take n $ (`mod`l)<$>randomIndices) xs)
takeSome :: Int -> [a] -> [a]
Take takeSome = takeSome'

In [6]:
import Linear.V2 (V2(..), _x, _y)

mapM_ print (
 linearRegressionW euclideanNorm
    (\v -> lfun $ \((a,b),c) -> a<.>^squareV v + b<.>^v + c)
 <$> [ takeSome n [ (V2 x y, 3*x + 2*y + x*y + 4 + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
     | n <- [5, 6, 10, 500]] )

((SymTensor {getSymmetricTensor = ex.⊗V2 (-5.008631851524115e-3) 0.4253389671849534 ^+^ ey.⊗V2 0.4253389671849534 0.6205986738204956},V2 3.473530098795891 (-2.34641695022583)),11.586738586425781)
((SymTensor {getSymmetricTensor = ex.⊗V2 1.0789454504447349e-2 0.4932505943033295 ^+^ ey.⊗V2 0.4932505943033295 (-4.003504907768729e-2)},V2 2.9914524398550384 2.3934176099137403),3.1007622114702826)
((SymTensor {getSymmetricTensor = ex.⊗V2 3.1091137636052313e-4 0.5103207479345682 ^+^ ey.⊗V2 0.5103207479345682 (-1.1255781164148715e-2)},V2 2.943478527740865 2.0121061168550227),3.9959890159668703)
((SymTensor {getSymmetricTensor = ex.⊗V2 (-5.8481528602882804e-5) 0.49975382298783827 ^+^ ey.⊗V2 0.49975382298783827 7.162427177043185e-5},V2 3.0026251474533012 2.0026078618996905),3.9861427747366633)

In [7]:
abc :: ((SymmetricTensor ℝ (V2 ℝ) +> ℝ, V2 ℝ +> ℝ), ℝ)
abc =
 linearRegressionW euclideanNorm
    (\v -> lfun $ \((a,b),c) -> (a $ squareV v) + (b $ v) + c)
  $ [ (V2 x y, 3*x + 2*y + x*y + 4 + δy)
       | x<-randomPoints
       , y<-randomPoints
       | δy<-randomDeviations ]
((a,b),c) = abc

In [8]:
import Control.Lens
import Math.LinearMap.Category.Derivatives

b ^. (1*∂id/∂_x)
b ^. (1*∂id/∂_y)

2.999798042719789

2.0009425298164274

In [9]:
ðxx, ðxy :: Lens' (V2 ℝ⊗〃+>ℝ) ℝ
ðxx = (1*∂id/∂_x.∂_x)
ðxy = (1*∂id/∂_x.∂_y)
a ^. ðxx
a ^. ðxy

5.7632660123374535e-5

0.9999159133245406

In [10]:
(a & ðxx .~ 4) ^. ðxx
(a & ðxx .~ 4) ^. ðxy

4.0

0.9999159133245406

In [11]:
let n = euclideanNorm :: Norm ℝ
 in mapM_ (\(m,dvs) -> print (m, convexPolytopeRepresentatives dvs :: [(ℝ,ℝ)]))
     $ linearRegressionWExtremeVar (\x -> lfun $ \(a,b) -> a*x + b)
     <$> [ [(-1,(0,n)), (0,(-1,n)), (0,(1,n)), (1,(0,n))]
         , [( 0,(0,n)), (1,(-1,n)), (1,(1,n)), (2,(0,n))]
         , [( 0,(0,n)), (1,(-1,n)), (1,(3,n)), (2,(0,n))]
         , [( 0,(0,n)), (1,(-1,n)), (1,(3,scaleNorm (sqrt 3/3) n)), (2,(0,n))]
         , [( 0,(0,n)), (0,(2,n)), (1,(1,n)), (1,(3,n))] ]

((0.0,0.0),[(0.0,2.0),(0.0,-2.0)])
((0.0,0.0),[(-4.2460344483505125e15,4.2460344483505145e15),(4.2460344483505125e15,-4.2460344483505145e15)])
((0.0,0.5),[(1.9611422589183607e-16,2.5),(3.843838827479988e-16,-3.5000000000000013)])
((0.0,0.0),[(-4.503599627370494e15,4.503599627370496e15),(1.2009599006321318e16,-1.2009599006321322e16)])
((1.0,1.0),[(-1.9999999999999998,2.0),(1.9999999999999998,-2.0),(1.9999999999999998,3.1401849173675503e-16),(-1.9999999999999998,-3.1401849173675503e-16)])

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [12]:
import Graphics.Dynamic.Plot.R2
import Data.Manifold.Shade ((|±|))

In [13]:
second convexPolytopeRepresentatives
 $ linearRegressionWExtremeVar (\x -> lfun $ \(c₀,(b₁,c₁)) -> c₀ + b₁*sin x + c₁*cos x)
                              (takeSome 30 [ (x, (sqrt 2 * sin (x + pi/4) + δy, spanNorm [1/0.05]))
                                           | x<-randomPoints
                                           | δy<-randomDeviations ])
    :: ((ℝ,(ℝ,ℝ)), [(ℝ,(ℝ,ℝ))])

((1.3007594251561025e-2,(0.9720688912324157,1.0220347394473663)),[(4.768054031313181e-2,(-5.842013611265602e-3,-9.441937157775972e-2)),(4.303334672581512e-2,(7.910492067963444e-2,1.9168376408331137e-2)),(2.9821750756779217e-2,(-5.5613170343463275e-2,5.585718634684596e-3)),(-2.5791847826019367e-2,(1.4504224650919328e-2,-5.83529013109633e-2)),(-4.720790007543039e-2,(-1.2307562626670662e-3,9.511646500792473e-2)),(5.3624962110344496e-2,(1.171784052362584e-2,0.11838202817489564)),(-4.5891010722877545e-2,(-6.785046937138133e-2,7.064585633746208e-2)),(-4.801120383560958e-2,(-9.133495028300118e-2,-7.349740330516589e-3)),(-2.7676664207006173e-2,(4.351969606823641e-2,-4.3991109301944664e-2))])

In [14]:
plotWindow [linregressionPlot (\x -> lfun $ \(c₀,(b₁,c₁)) -> c₀ + b₁*sin x + c₁*cos x)
                              (takeSome 30 [ (x, sqrt 2 * sin (x + pi/4) + δy |±|[0.05])
                                           | x<-randomPoints
                                           | δy<-randomDeviations ])
                              (const (<>)) ]

GraphWindowSpecR2{lBound=-1.3892143047161305, rBound=11.459961691577028, bBound=-1.8948412282702722, tBound=1.9993244364320613, xResolution=640, yResolution=480}

![Plot of regression of a set of deviated sine-sampled function points, with uncertainties.](https://raw.githubusercontent.com/leftaroundabout/linearmap-family/master/test/images/regression-envi.png)

In [15]:
plotWindow [linregressionPlot (\x -> lfun $ \(c,(b,a)) -> a*x^2 + b*x + c)
                              (takeSome 30 [ (x, x*(x - 3)/100 + δy |±|[0.05])
                                           | x<-(+8)<$>randomPoints
                                           | δy<-randomDeviations ])
                              (const (<>)) ]

GraphWindowSpecR2{lBound=-3.946526318751707, rBound=28.320733910851708, bBound=-5.782904275005686, tBound=7.283322977436324, xResolution=640, yResolution=480}

In [16]:
x¦y = (x :: ℝ, (y::ℝ)|±|[0.5])
plotWindow [linregressionPlot (\x -> lfun $ \(c,(b,a)) -> a*x^2 + b*x + c)
                              [ 0¦0, 0¦1, 1¦1, 1¦2, 2¦4, 2¦5 ]
                              (const (<>)) ]

GraphWindowSpecR2{lBound=-0.3333333333333333, rBound=2.333333333333333, bBound=-1.5, tBound=6.5, xResolution=640, yResolution=480}